In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('creditcard_sampledata_2.csv')
data.head()

,Unnamed: 0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,221547,-1.191668,0.428409,1.640028,-1.848859,-0.870903,-0.204849,-0.385675,0.352793,-1.098301,...,0.507173,1.292565,-0.467752,1.244887,0.697707,0.059375,-0.319964,-0.017444,27.44,0
1,184524,1.966614,-0.450087,-1.228586,0.142873,-0.150627,-0.543590,-0.076217,-0.108390,0.973310,...,0.124055,0.564916,-0.039331,-0.283904,0.186400,0.192932,-0.039155,-0.071314,35.95,0
2,91201,1.528452,-1.296191,-0.890677,-2.504028,0.803202,3.350793,-1.633016,0.815350,-1.884692,...,-0.412088,-1.017485,0.129566,0.948048,0.287826,-0.396592,0.042997,0.025853,28.40,0
3,26115,-0.774614,1.100916,0.679080,1.034016,0.168633,0.874582,0.209454,0.770550,-0.558106,...,0.011158,0.146017,-0.130401,-0.848815,0.005698,-0.183295,0.282940,0.123856,43.20,0
4,201292,-1.075860,1.361160,1.496972,2.242604,1.314751,0.272787,1.005246,0.132932,-1.558317,...,0.254485,0.530692,-0.651119,0.626389,1.040212,0.249501,-0.146745,0.029714,10.59,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7300 entries, 0 to 7299
Data columns (total 31 columns):
Unnamed: 0    7300 non-null int64
V1            7300 non-null float64
V2            7300 non-null float64
V3            7300 non-null float64
V4            7300 non-null float64
V5            7300 non-null float64
V6            7300 non-null float64
V7            7300 non-null float64
V8            7300 non-null float64
V9            7300 non-null float64
V10           7300 non-null float64
V11           7300 non-null float64
V12           7300 non-null float64
V13           7300 non-null float64
V14           7300 non-null float64
V15           7300 non-null float64
V16           7300 non-null float64
V17           7300 non-null float64
V18           7300 non-null float64
V19           7300 non-null float64
V20           7300 non-null float64
V21           7300 non-null float64
V22           7300 non-null float64
V23           7300 non-null float64
V24           7300 non-null flo

In [4]:
n_fraud = data.Class.value_counts()
n_fraud

0    7000
1     300
Name: Class, dtype: int64

In [5]:
natural_accuracy = n_fraud.iloc[0]/data.Class.count()
natural_accuracy

0.958904109589041

In [6]:
X = data.iloc[:,1:-1].values
y = data.iloc[:,-1].values

In [7]:
from imblearn.over_sampling import SMOTE

In [8]:
method = SMOTE(kind='regular')

In [9]:
X_resampled, y_resampled = method.fit_sample(X, y)
X_resampled.shape

(14000, 29)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2)

In [11]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

C:\Users\sr4rish\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [38]:
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators': [60, 80], 'max_features': ['auto', 'log2'], 'criterion': ['gini', 'entropy']
}
grid_search = GridSearchCV(estimator=classifier, param_grid=parameters, cv=10, scoring='precision', n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [60, 80], 'max_features': ['auto', 'log2'], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='precision', verbose=0)

In [37]:
grid_search.best_params_

{'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 60}

In [36]:
grid_search.best_score_

0.9976529290357765

In [34]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
print('confusion matrix:\n',cm,'\n\n','classification report:\n',cr,'\n\n','Accucary:\n',acc)

confusion matrix:
 [[1350    5]
 [  23 1422]] 

 classification report:
              precision    recall  f1-score   support

          0       0.98      1.00      0.99      1355
          1       1.00      0.98      0.99      1445

avg / total       0.99      0.99      0.99      2800
 

 Accucary:
 0.99
